<a href="https://colab.research.google.com/github/sammanfatima/Pytorch-Journey/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Import Libraries***

In [50]:
import pandas as pd
import numpy as numpy
import re
import string
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import streamlit as st

# ***Load Dataset***

In [51]:
df = pd.read_csv('/content/drive/MyDrive/Pytorch Learning/news.csv')

In [52]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [54]:
df.describe

<bound method NDFrame.describe of       Unnamed: 0  ... label
0           8476  ...  FAKE
1          10294  ...  FAKE
2           3608  ...  REAL
3          10142  ...  FAKE
4            875  ...  REAL
...          ...  ...   ...
6330        4490  ...  REAL
6331        8062  ...  FAKE
6332        8622  ...  FAKE
6333        4021  ...  REAL
6334        4330  ...  REAL

[6335 rows x 4 columns]>

# ***Handle Labels***

In [55]:
df['label'] = df['label'].map({'FAKE': 0, 'REAL': 1})

# ***Text Cleaning***

In [56]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'\[.*?\]', '', text)
  text = re.sub(r'https?://\S+|WWW\.\S+', '', text)
  text = re.sub(r'<.*?>+', '', text)
  text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'\w*\d\w*', '', text)
  return text

In [19]:
df['tex'] =df['text'].apply(clean_text)

# ***Tf-IDF Vectorization***

In [57]:
X = df['text']
y = df['label']
tfidf=TfidfVectorizer(max_features=5000)
X= tfidf.fit_transform(X).toarray()

In [58]:
print(X)
print(y)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.00794729 0.         ... 0.01825211 0.         0.        ]
 [0.         0.01684576 0.0164676  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64


# ***Train and test split***

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ***Convert to Pytroch Tensors***

In [60]:
X_train = torch.tensor(X_train, dtype= torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# ***Neural Network Model***

In [61]:
class FakeNewsNN(nn.Module):
    def __init__(self, input_size):
        super(FakeNewsNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) # first hidden layer
        self.fc2 = nn.Linear(128, 64) # second hidden layer
        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


# ***Initialize Model***

In [62]:
model = FakeNewsNN(X_train.shape[1])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# ***Training Loop***

In [63]:
epochs = 10

for epoch in range(epochs):
    model.train()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.6932
Epoch [2/10], Loss: 0.6918
Epoch [3/10], Loss: 0.6899
Epoch [4/10], Loss: 0.6874
Epoch [5/10], Loss: 0.6841
Epoch [6/10], Loss: 0.6803
Epoch [7/10], Loss: 0.6760
Epoch [8/10], Loss: 0.6710
Epoch [9/10], Loss: 0.6655
Epoch [10/10], Loss: 0.6595


# ***Evaluation***

In [64]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = (y_pred > 0.5).int()

accuracy = accuracy_score(y_test, y_pred_class)
print(f"\nAccuracy: {accuracy*100:.2f}%")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_class))



Accuracy: 89.27%

Classification Report:

              precision    recall  f1-score   support

         0.0       0.92      0.85      0.89       628
         1.0       0.87      0.93      0.90       639

    accuracy                           0.89      1267
   macro avg       0.90      0.89      0.89      1267
weighted avg       0.89      0.89      0.89      1267



# ***Prediction***

In [65]:
def predict_news(text):
    text = clean_text(text)
    vector = tfidf.transform([text]).toarray()
    vector = torch.tensor(vector, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        prediction = model(vector)

    return "REAL NEWS" if prediction.item() > 0.5 else "FAKE NEWS"



In [66]:
print(predict_news("Breaking: Government announces new tax reforms"))

FAKE NEWS
